# Black-Owned Businesses in Boston

## Importing Dependencies

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
import folium
import time
import warnings
warnings.filterwarnings('ignore')

## Restaurants

In [2]:
restaurant = pd.read_csv('/Users/avonleafisher/Desktop/Restaurants.csv')

In [3]:
restaurant = restaurant.loc[3:]
header = restaurant.iloc[0] 
restaurant = restaurant[1:] 
restaurant.columns = header
restaurant.drop('#', axis=1, inplace=True)
restaurant = restaurant[restaurant['Street Address'].isna() == False]

In [4]:
restaurant.head()

3,Name,Black Owned Confirmed?,Area,Direct Venmo/Donation Option/Giftcard,Cuisine/ Shop type,Website,Instagram,FB,Twitter,YouTube,Delivery Options,Combined Address,Street Address,City,State,Zipcode,Email Address,Phone
4,Rock City Pizza,Confirmed,Allston,NaN,American,https://rockcitypizzeria.com/,https://www.instagram.com/rockcitypizzaallston/,https://www.facebook.com/RockCityPizzeria/,NaN,NaN,NaN,"568 Cambridge St, Boston, MA 02134",568 Cambridge St,Allston,MA,02134,NaN,NaN
5,TamBos Kitchen,Unconfirmed,Avon,NaN,West African,https://tamboskitchen.com/,https://www.instagram.com/tamboskitchen/,https://www.facebook.com/tamboskitchen/,NaN,https://www.youtube.com/channel/UCfh-OI9DB3Wtm...,Local,"490 W Main St, Avon, MA 02322",490 W Main St,Avon,MA,02322,NaN,NaN
6,Restaurante Cesaria,Confirmed,Boston,NaN,African/ Cape Verdean,https://www.restaurantcesaria.com/,https://www.instagram.com/restaurantecesaria/,https://www.facebook.com/RestaurantCesaria/,NaN,NaN,"Doordash, grubhub, ubereats","266 Bowdoin St, Boston, MA 02122",266 Bowdoin St,Boston,MA,02122,NaN,NaN
7,Bred Gourmet,Confirmed,Boston,http://www.bredgourmet.com/shopbred/bred-gift-...,American,http://www.bredgourmet.com/,https://www.instagram.com/bredgourmet/,https://www.facebook.com/BREDGOURMET/,https://twitter.com/BredGourmet,NaN,Grubhub,"2255 Dorchester Ave, Boston, MA 02124",2255 Dorchester Ave,Boston,MA,02124,NaN,6176980103
8,Darryl's Corner Bar and Kitchen,Confirmed,Boston,https://www.toasttab.com/darryls-corner-bar-ki...,American,http://dcbkboston.com/,https://www.instagram.com/dcbkboston/,https://www.facebook.com/DCBKBoston/,https://twitter.com/DCBKBoston,NaN,Grubhub,"604 Columbus Ave, Boston, MA 02118",604 Columbus Ave,Boston,MA,02118,NaN,6175361100


In [5]:
areas = list(restaurant.Area.unique())

In [6]:
restaurant.reset_index(inplace=True)

In [7]:
restaurant['Street Address'] = [i.title() for i in restaurant['Street Address']]

In [8]:
restaurant['Street Address'][94] = "5197 Washington Street"
restaurant['Street Address'][53] = "233 River Street"
restaurant['Street Address'][89] = "285 Beacon Street"

In [9]:
restaurant['Street Address'] = [i + ',' for i in restaurant['Street Address']]
restaurant['City'] = [str(i) + ',' for i in restaurant['City']]
restaurant['Address'] = restaurant['Street Address'].str.cat(restaurant[['City', 'State', 'Zipcode']], sep=' ')
restaurant = restaurant[['Name', 'Address', 'Website']]

In [10]:
restaurant.head()

3,Name,Address,Website
0,Rock City Pizza,"568 Cambridge St, Allston, MA 02134",https://rockcitypizzeria.com/
1,TamBos Kitchen,"490 W Main St, Avon, MA 02322",https://tamboskitchen.com/
2,Restaurante Cesaria,"266 Bowdoin St, Boston, MA 02122",https://www.restaurantcesaria.com/
3,Bred Gourmet,"2255 Dorchester Ave, Boston, MA 02124",http://www.bredgourmet.com/
4,Darryl's Corner Bar and Kitchen,"604 Columbus Ave, Boston, MA 02118",http://dcbkboston.com/


In [11]:
geolocator = Nominatim(user_agent='fisheravonlea@gmail.com')

In [12]:
basemap = folium.Map(location=[42.361145, -71.057083], tiles="Stamen Toner", zoom_start=10)

In [13]:
def get_lat_lon(address_column):
    
    lat = []
    lon = []
    
    for a in address_column:
        location = geolocator.geocode(a)
        
        if location is not None:
            lat.append(location.latitude)
            lon.append(location.longitude)
        else:
            lat.append('NA')
            lon.append('NA')
    return lat, lon

In [14]:
def add_markers(df, color, icon):
    
    df['Lat'], df['Lon'] = get_lat_lon(df.Address)
    df = df[df.Lat != 'NA']
    
    points = list(zip(df.Lat, df.Lon, df.Name, df.Address, df.Website))
    
    for p in points:
        
        if pd.isnull(p[4]):
            
            popup_text = "{}, {}".format(p[2], str(p[3]))
        
            popup = folium.Popup(popup_text, 
                             autopan='False',
                             parse_html=True)
                
            marker = folium.Marker(location=[p[0], p[1]], 
                                popup=popup, 
                                icon = folium.Icon(icon_size=(27, 27),
                                                      color=color,
                                                      icon=icon,
                                                      prefix='fa'),
                                    ).add_to(basemap)
            
        else:
            
            popup_text = "{}, {}, {}".format(p[2], str(p[3]), p[4])
        
            popup = folium.Popup(popup_text, 
                             autopan='False',
                             parse_html=True)
                
            marker = folium.Marker(location=[p[0], p[1]], 
                                popup=popup, 
                                icon = folium.Icon(icon_size=(27, 27),
                                                      color=color,
                                                      icon=icon,
                                                      prefix='fa'),
                                    ).add_to(basemap)

In [15]:
add_markers(restaurant, 'green', 'cutlery')

In [16]:
store = pd.read_csv('/Users/avonleafisher/Desktop/Stores.csv')

In [17]:
header = store.iloc[0] 
store = store[1:] 
store.columns = header

In [18]:
store = store[['NAME', 'WEBSITE', 'ADDRESS']]
store.columns = [c.title() for c in store.columns]
store = store.dropna()

In [19]:
store.head()

,Name,Website,Address
1,RepHAIRations,https://www.rephairations.com/,"1339 Massachusetts Avenue Arlington, MA 02476"
2,Mamadou' Artisan Bakery,https://www.yelp.com/biz/mamadous-artisan-bake...,"677 Massachusetts Ave Arlington, MA 02476 Arli..."
3,Elite Lash Studio,http://elitelashstudio.com,"224 Clarendon Street, Suite 51 Boston, MA 02116"
5,Bebe Mursalin,https://bebemursalin.com/,"32 Province Street, Boston, MA 02108"
6,Boston Beauty Bar,http://bostonbeautybar.as.me/,"388 Washington St, Boston, MA 02108"


In [20]:
add_markers(store, 'beige', 'usd')

## Services

In [21]:
service = pd.read_csv('/Users/avonleafisher/Desktop/Services.csv')

In [22]:
header = service.iloc[0] 
service = service[1:] 
service.columns = header

In [23]:
service = service[service.City.isin(areas)]
service['Address'] = [str(i) + ',' for i in service['Address']]
service['City'] = [str(i) + ',' for i in service['City']]
service['Address'] = service['Address'].str.cat(service[['City', 'State', 'Zip code']], sep=' ')

In [24]:
service = service[['Company Name', 'Address', 'Category', 'Website']]
service = service.rename(columns={'Company Name': 'Name'}).drop_duplicates(
                                                                           subset='Name', 
                                                                           keep="first").dropna(
                                                                           subset=['Address'])

In [25]:
service.head()

,Name,Address,Category,Website
30,"Atlantic Bay Contracting Co., Inc.","100 Hano Street, Suite 22, Allston, MA 02134",AB - ASBESTOS ABATEMENT,NaN
32,Fulltec Consultants,"1399 Commonwealth Avenue, Allston, MA 02134",CP3 - COMPUTER NETWORKING,NaN
35,"Paragon Global Partners, Inc.","100 Hano Street, Suite 10, Allston, MA 02134",EN4 - LEAD PAINT SERVICES,http://www.pgpi.us.com
38,"RTD Logistics, LLC d/b/a: Skycom Courier","119 Braintree Street, Suite 101, Allston, MA 0...",TR0 - COURIER SERVICES,http://www.skycomcourier.com
90,"Asaman, Inc.","258 Bodwell Street, Avon, MA 02322",MW - MERCHANT WHOLESALES,http://www.asaman.com


In [26]:
health = service[service.Category.isin(['HS - HEALTH CARE SERVICES AND CONSULTANTS', 
                                        'HA - HEALTH CARE SERVICES (NON-PROFIT)'])]

In [27]:
add_markers(health, 'red', 'stethoscope')

In [28]:
computer = service[service.Category.isin(['CP7 - COMPUTER TRAINING, SUPPORT SERVICES AND CONSULTING', 
                                          'CP - COMPUTER SOFTWARE',
                                          'CP1 - COMPUTER INTEGRATION', 
                                          'CP9 - INFORMATION TECHNOLOGY',
                                          'CP3 - COMPUTER NETWORKING',
                                          'CP6 - COMPUTER PROGRAMMING',
                                          'CP10 - COMPUTER DESIGN'])]
add_markers(computer, 'blue', 'laptop')

In [29]:
cleaning = service[service.Category.isin(['MA1 - JANITORIAL CLEANING',
                                          'EN11 - WASTE MANAGEMENT AND REMEDIATION SERVICES'])]
add_markers(cleaning, 'orange', 'trash')

In [30]:
home = service[service.Category.isin([
             'HB - HOME BUILDING CONTRACTORS',
             'HIC - HOME IMPROVEMENT CONTRACTOR',
             'RC - ROOFING CONTRACTORS',
             'FL - FLOORING CONTRACTORS',
             'PL - PLASTERING AND DRYWALL CONTRACTORS',
             'CM1 - MASONRY CONTRACTORS',
             'CA - CARPENTRY CONTRACTORS',
             'IN - INTERIOR DEMOLITION'])]
add_markers(home, 'black', 'home')

In [31]:
architects = service[service.Category == 'AE - ARCHITECTS']
add_markers(architects, 'cadetblue', 'building')

In [32]:
marketing = service[service.Category == 'MK - MARKETING']
add_markers(marketing, 'lightgray', 'bullhorn')

In [33]:
nonprofit = service[service.Category.isin([
             'MO - MULTI-SERVICE ORGANIZATIONS (NON-PROFIT)',
             'ED - EDUCATIONAL SERVICES (NON-PROFIT)',
             'HO - HOUSING ADVOCACY (NON-PROFIT)',
             'CD - CHILDCARE (NON-PROFIT)',
             'EC - ECONOMIC DEVELOPMENT (NON-PROFIT)',
             'ET - ELDERLY SERVICES (NON-PROFIT)',
             'ST - SHELTER SERVICES (NON-PROFIT)'])]
add_markers(nonprofit, 'purple', 'heart')

In [34]:
basemap

In [35]:
basemap.save('businesses.html')